# File Description 
* Here, I am going to iterate through all of the annotations for each of the SCC files (not yet those that have dot annotations, but only those that do nuclei contour annotations 
* I'm going to get the bounding box, splice that portion of the WSI, resize it to 32 x 32, and then save it as an npy file. 
    * Each file should be named sample_id_[nuclei_num].npy 

* While doing this, I'm going to preserve a metadata dict, where each row will include
    * file path 
    * label 
    * xmin, xmax, ymin, ymax
    * height
    * width 
    

In [1]:
import pandas as pd
import numpy as np
import torch 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
import glob, os, pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch import nn
from matplotlib import cm
import cv2

/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import PIL 
from PIL import Image as im
from IPython.display import Image 
import xmltodict as xd

In [3]:
import tifffile
from pathpretrain.utils import load_image
from pathpretrain.process_xml import process_xml

In [4]:
base_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/scc_nuclei/data/nuclei_contour_annotations/"

#all of the xml files with annotations
xml_files = [
    "327_A1d_ASAP.xml",
    "281_A1f_ASAP.xml",
    "270_A1d_ASAP.xml",
#     "350 A1b_ASAP.xml", 
#     "355 A1d_ASAP.xml", 
    
]

#the actual image files
img_files = [
    "327_A1d_ASAP.tif",
    "281_A1f_ASAP.tif",
    "270_A1d_ASAP.tif",
#     "350 A1b_ASAP.tif", 
#     "355 A1d_ASAP.tif", 
]
#here we will map 
annotations = {}
imgs = {}

## Read in all the images/annotations

In [5]:
PIL.Image.MAX_IMAGE_PIXELS = 3266158455

In [6]:
for f,i in tqdm(zip(xml_files, img_files)): 
    annotations[f] = process_xml(base_dir+f, transpose = False)['contour']
    imgs[i] = np.asarray(im.open(base_dir+i))

0it [00:00, ?it/s]/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[0][7]=x[0][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[1][7]=x[1][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[0][6]=x[0][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[1][6]=x[1][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Sriniva

/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[0][14]=x[0][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[1][14]=x[1][0]
  (i, m, i)))
3it [01:38, 32.86s/it]


In [22]:
annotations["327_A1d_ASAP.xml"]

,contours,xmin,xmax,ymin,ymax,xmean,ymean,lbl
0,"[[19183.445299999992, 18803.369099999996], [19...",19166.5527,19183.4453,18793.3867,18809.1270,19174.176057,18801.394800,scc nuclei
1,"[[19184.595699999998, 18784.173799999997], [19...",19166.9355,19191.5059,18781.1016,18797.2266,19179.861000,18790.380217,scc nuclei
2,"[[19300.539100000005, 18805.287100000005], [19...",19299.7715,19320.1172,18792.2363,18811.4316,19308.107143,18802.546029,scc nuclei
3,"[[19326.644500000002, 18813.3496], [19326.6964...",19315.1270,19331.6348,18810.6641,18830.2422,19323.402578,18820.730044,scc nuclei
4,"[[19335.089799999994, 18801.0664], [19335.0719...",19335.0898,19348.5273,18796.0742,18812.5840,19340.904000,18804.027343,scc nuclei
...,...,...,...,...,...,...,...,...
1349,"[[18192.4355, 20206.636699999992], [18192.3949...",18189.3574,18206.6660,20206.6367,20228.5605,18197.947900,20217.662750,scc nuclei
1350,"[[18206.281300000002, 20191.2539], [18206.3233...",18192.4355,18207.0508,20188.5605,20208.1758,18200.319983,20198.304350,scc nuclei
1351,"[[18122.4355, 20192.021499999995], [18122.4001...",18115.1289,18127.8203,20192.0215,20208.1758,18121.282233,20200.418967,scc nuclei
1352,"[[17972.4375, 20183.945300000003], [17972.4779...",17957.0547,17974.7461,20179.7148,20196.2520,17966.174671,20188.176071,scc nuclei


## Create metadata df

In [68]:
metadata = pd.DataFrame()
# data 
filepaths = []
samples = []
labels = []
heights = []
widths = []
dimensions = [] #(xmin, ymin, xmax, ymax)

In [69]:
# label == "inflammatory cells" or label == "inflammatory nuclei"
# label == "scc nuclei":

# Iterate through images

In [70]:
from matplotlib import cm
def resize_img(img):
    img = im.fromarray(img.astype('uint8'), 'RGB')
    img = img.resize((32, 32))
    img = np.asarray(img)
#     plt.imshow(img)
#     plt.show(img)
    return img

In [71]:
base_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/Nuclei_Classification/data/"

In [72]:
for id in annotations: 
    sample_id = id.split(".")[0]
    WSI = imgs[str(sample_id + ".tif")]
    idx = 0 
    for c, lbl, xmin, xmax, ymin, ymax in tqdm(zip(annotations[id]["contours"], annotations[id]["lbl"], annotations[id]["xmin"], annotations[id]["xmax"], annotations[id]["ymin"], annotations[id]["ymax"])):
        #get the image and resize it
        small_img = WSI[int(ymin):int(ymax) + 1, int(xmin):int(xmax) + 1]
        small_img = resize_img(small_img)
        #get label 
        label = 1
        if lbl == "scc nuclei":
            label = 1
        elif lbl == "inflammatory cells" or lbl == "inflammatory nuclei":
            label = 0
        #save image 
        path = base_dir + sample_id + "_" + str(idx) + ".npy"
        np.save(path, small_img)
        
        #now append all of the arrays with correct information 
        filepaths.append(path)
        labels.append(label)
        heights.append(32)
        widths.append(32)
        dimensions.append((xmin, ymin, xmax, ymax))
        samples.append(sample_id)
        idx += 1

1354it [00:10, 131.82it/s]
1259it [00:09, 136.48it/s]
1138it [00:10, 104.48it/s]


In [74]:
metadata["paths"] = filepaths 
metadata["sample"] = samples 
metadata["labels"] = labels 
metadata["heights"] = heights 
metadata["widths"] = widths 
metadata["dimensions"] = dimensions
metadata.to_csv(str(base_dir + "metadata.csv"))

In [75]:
metadata

,paths,sample,labels,heights,widths,dimensions
0,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,327_A1d_ASAP,1,32,32,"(19166.5527, 18793.3867, 19183.4453, 18809.127)"
1,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,327_A1d_ASAP,1,32,32,"(19166.9355, 18781.1016, 19191.5059, 18797.2266)"
2,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,327_A1d_ASAP,1,32,32,"(19299.7715, 18792.2363, 19320.1172, 18811.4316)"
3,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,327_A1d_ASAP,1,32,32,"(19315.127, 18810.6641, 19331.6348, 18830.2422)"
4,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,327_A1d_ASAP,1,32,32,"(19335.0898, 18796.0742, 19348.5273, 18812.584)"
...,...,...,...,...,...,...
3746,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,270_A1d_ASAP,0,32,32,"(11497.5039, 18522.9902, 11504.8701, 18536.8008)"
3747,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,270_A1d_ASAP,1,32,32,"(11350.1709, 18447.0195, 11373.6514, 18464.9766)"
3748,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,270_A1d_ASAP,1,32,32,"(11365.8242, 18429.0645, 11388.8457, 18456.2285)"
3749,/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Stud...,270_A1d_ASAP,0,32,32,"(11081.5391, 18514.9668, 11092.8975, 18526.7793)"
